In [16]:
import pandas as pd

In [17]:
eth = pd.read_csv('eth.csv')

In [18]:
eth['movingAverage'] = eth.adjClose.rolling(20).mean()
eth['standardDeviation'] = eth.adjClose.rolling(20).std()

In [19]:
eth['BollingerBandsUpper'] = eth['movingAverage'] + (eth['standardDeviation'] * 2)
eth['BollingerBandsLower'] = eth['movingAverage'] - (eth['standardDeviation'] * 2)

In [20]:
eth.to_csv('eth_1.csv', index=False)